# Tâche 1 - Extraction d'informations à partir de recettes

Construisez des expressions régulières pour repérer les aliments et les quantités de chaque item du fichier ***data/t1_ingredients.txt***. Compléter la fonction ***get_ingredients*** de la Section 3 de ce *notebook* afin de retourner la quantité et l’ingrédient d’un item. Par exemple, 


    get_ingredients("2 cuillères à café de poudre à pâte")

devrait retourner la paire :

    "2 cuillères à café", "poudre à pâte"

Par la suite, présentez à la Section 4 les performances que vous obtenez avec ces expressions régulières sur le fichier ***data/t1_test.json***. Discutez des principales erreurs commises par vos expressions régulières. 

## Section 1 - Lecture du fichier pour la construction d'expressions régulières

In [1]:
ingredients_fn = "./data/t1_ingredients.txt"

In [2]:
def load_ingredients(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        raw_items = f.readlines()
    ingredients = [x.strip() for x in raw_items]
    return ingredients

In [3]:
ingredients = load_ingredients(ingredients_fn)
len(ingredients)

50

In [4]:
ingredients

['0,5 tasse sucre',
 '2 bananes en purée',
 '2 oeufs',
 '2 tasses farine',
 '2 cuillères à café poudre à pâte',
 '0,5 cuillère à café sel',
 '1 cuillère à café poudre à pâte',
 '6 pommes vertes pelées, en tranches',
 'sel',
 '2 cuillères à soupe jus de citron',
 '3 échalotes coupées finement',
 '0,75 tasse vin blanc sec',
 '2 tasses crème 35%',
 '1 pincée de muscade',
 '1 lb poitrine de poulet désossées et coupées en petits morceaux',
 '1 oeuf battu',
 '0,5 cuillère à café (2 ml) sel',
 '1 cuillère à soupe (15 ml) gingembre râpé',
 '0,25 tasse (65 ml) oignon vert coupés en biseaux',
 '0,25 tasse (65 ml) eau ou au goût',
 '3 tasses pommes de terre en purée',
 '0,25 tasse échalote hachées',
 '2 boîtes de conserve (213 g) saumons',
 "1 enveloppe soupe à l'oignon",
 '1 rôti de 675 g (1 ½ lb) de filet mignon de bœuf (prélevé dans le centre du filet)',
 '30 ml (2 c. à soupe) d’huile d’olive',
 '190 g (¾ tasse) de beurre non salé',
 '7,5 ml (1 ½ c. à thé) d’estragon frais ciselé',
 '6 oignons

Les résultats attendus des 50 exemples utilisés pour la construction de vos expressions régulières sont: 

In [5]:
import json 

solution_fn = 'data/t1_ingredients_solution.json'

with open(solution_fn, 'r', encoding='utf-8') as fp:
    solutions = json.load(fp)

len(solutions)

50

In [ ]:
solutions

[{'text': '0,5 tasse sucre', 'quantity': '0,5 tasse', 'ingredient': 'sucre'},
 {'text': '2 bananes en purée', 'quantity': '2', 'ingredient': 'bananes'},
 {'text': '2 oeufs', 'quantity': '2', 'ingredient': 'oeufs'},
 {'text': '2 tasses farine', 'quantity': '2 tasses', 'ingredient': 'farine'},
 {'text': '2 cuillères à café de poudre à pâte',
  'quantity': '2 cuillères à café',
  'ingredient': 'poudre à pâte'},
 {'text': '0,5 cuillère à café sel',
  'quantity': '0,5 cuillère à café',
  'ingredient': 'sel'},
 {'text': '1 cuillère à café poudre à pâte',
  'quantity': '1 cuillère à café',
  'ingredient': 'poudre à pâte'},
 {'text': '6 pommes vertes pelées, en tranches',
  'quantity': '6',
  'ingredient': 'pommes vertes'},
 {'text': 'sel', 'quantity': '', 'ingredient': 'sel'},
 {'text': '2 cuillères à soupe jus de citron',
  'quantity': '2 cuillères à soupe',
  'ingredient': 'jus de citron'},
 {'text': '3 échalotes coupées finement',
  'quantity': '3',
  'ingredient': 'échalotes'},
 {'text': 

## Section 2 - Vos expressions régulières

Mettez dans cette section toutes les expressions régulières que vous avez construites à partir du fichier ***data/t1_ingredients.txt***. 
Vous pouvez ajouter des cellules dans cette section du *notebook* si cela permet de rendre la description plus claire. 

In [42]:
import re

# Vos expressions régulières
# Expression régulière pour récupérer les quantités.
quantity_regex = r'((\(?((\d+,\d*)|(\d+ à \d+)|(\d (⅓|½|¾))|\d+/\d+|\d+(?!\d+)(?!%)|(⅓|½|¾)) {0,1}(kg|ml|g(?!\w)|c\. à c\.|c\. à soupe|c\. à \.s|c\. à s\.|cm|oz|po(?!\w)|lb|c\. à thé|tasse(s?)|Bouquet(s?)|enveloppe(s?)|cuillère(s?) à soupe|cuillère(s?) à café|gousse(s?)|tranche(s?)|botte(s?)|boîte(s?) de conserve|pincée(s?))?\)?|au goût|Quelque(s?) sommité(s?)) ?)+'
quantity_regex = re.compile(quantity_regex, re.IGNORECASE)

# Expression régulière pour récupérer les ingrédients après avoir enlevé les quantités.
food_regex = r"(\w{3,} +[\w|’|']+ +[\w|’|']+)|(\w{3,} +[\w|’|'|%]+)|(\w{3,})"
food_regex = re.compile(food_regex, re.IGNORECASE)


L'expression pour récuperer les quantités est relativement complexe, elle récupère tous les chiffres décimaux ou non et les fractions écrites en un charactère tel que ⅓ et sur plusieurs charactères tel que 1/3. Ensuite, elle capture les termes suivants basé sur un liste prédéfinie tel que c. à s. ou cuil`ere à soupe.
Le regex pour des ingrédients capture la première séquence de un à trois termes avec le premier terme ayant au moins 3 lettres suivant les quantités. 
Une explication plus détaillée se trouve ci-dessous dans la partie "Analyse des résultats".

## Section 3 - Fonction pour l'extraction des ingrédients

La fonction principale ***get_ingredients*** prend en entrée une ligne de texte et retourne une paires de valeurs: la quantité et l'aliment.

Vous pouvez ajouter autant de sous-fonctions que vous le souhaitez.
Il est cependant important de ne pas modifier la signature de la fonction principale afin de faciliter notre travail de correction.

In [ ]:
def get_ingredients(text):
    quantity_match = quantity_regex.match(text)
    if quantity_match is not None:
        quantity = quantity_match.group().strip()
    else:
        quantity = ""
    # On retire les quantités de la ligne pour simplifier la recherche des ingrédients
    aliments = re.sub(quantity_regex,'',text).strip()
    food_match = food_regex.search(aliments)
    if food_match is not None:
        food = food_match.group().strip()
    else:
        food = ""
    return quantity, food 

In [44]:
for ingredient in ingredients:
    quantity, food = get_ingredients(ingredient)
    # print(f"Quantity: {quantity}, Food: {food}")

Vous pouvez mettre ici tout commentaire qui nous aiderait à comprendre votre fonction et pour expliquer votre démarche. De plus, indiquez si vous réussissez à extraire toutes les informations du fichier d'ingrédients. Sinon, donnez des précisions.  

La fonction premièrement cherche les quantités de chaque ligne de texte. Pour recupérer les aliments, les quantités sont retirés et le reste est cherché avec la expression régulière.

## Section 4 - Évaluation et analyse de vos résultats

Décrivez ici les résultats obtenus et présentez l'évaluation obtenue sur le fichier de test ***data/t1_test.json***. Présentez des exemples d'erreurs. Vous pouvez ajouter le nombre de cellules que vous souhaitez pour faire votre analyse. 

Dans un premier temps, on monte en mémoire les exemples de tests. Vous deviez en avoir 26. Chacun contient le texte de l'ingrédient, la quantité et l'aliment. 

In [45]:
import json

def load_test_set(filename):
    with open(filename, 'r', encoding='utf-8') as fp:
        tests = json.load(fp)
    return tests

In [46]:
test_fn = "./data/t1_test.json"
# test_examples
test_examples = load_test_set(test_fn)
len(test_examples)

126

Les prochaines cellules présentent l'évaluation (dont le code pour mener l'évaluation) et votre analyse des résutlats. 

In [48]:
total_quantity = 0
total_food = 0

for ingredient in test_examples:
    quantity, food = get_ingredients(ingredient["text"])
    same_quantity = 1 if quantity == ingredient["quantity"] else 0
    total_quantity += same_quantity
    same_food = 1 if food == ingredient["ingredient"] else 0
    total_food += same_food
    if same_food == 0 or same_quantity == 0:
        print(f"{same_quantity} {same_food} Quantity: {quantity}|{ingredient['quantity']} --- Food:{food}|{ingredient['ingredient']}")

print(f"Quantity: {total_quantity}/{len(test_examples)}")
print(f"Ingredient: {total_food}/{len(test_examples)}")

1 0 Quantity: 2|2 --- Food:bananes en purée|bananes
1 0 Quantity: 6|6 --- Food:pommes vertes pelées|pommes vertes
1 0 Quantity: 3|3 --- Food:échalotes coupées finement|échalotes
1 0 Quantity: 8|8 --- Food:champignons émincés|champignons
1 0 Quantity: 1|1 --- Food:oeuf battu|oeuf
1 0 Quantity: 1 cuillère à soupe (15 ml)|1 cuillère à soupe (15 ml) --- Food:gingembre râpé|gingembre
1 0 Quantity: 0,25 tasse (65 ml)|0,25 tasse (65 ml) --- Food:oignon vert coupés|oignon vert
0 0 Quantity: 0,25 tasse (65 ml)|0,25 tasse (65 ml) ou au goût --- Food:eau ou|eau
0 0 Quantity: 0,25 tasse (65 ml)|0,25 tasse (65 ml) ou au goût --- Food:sucre ou|sucre
0 1 Quantity: 1 cuillère à soupe (15 ml)|cuillère à soupe (15 ml) --- Food:fécule de maïs|fécule de maïs
1 0 Quantity: 0,5 tasse|0,5 tasse --- Food:fromage cheddar râpé|fromage cheddar
1 0 Quantity: 0,25 tasse|0,25 tasse --- Food:échalote hachées|échalote
1 0 Quantity: 1|1 --- Food:oeuf battu|oeuf
0 0 Quantity: 1|1 rôti de 675 g (1 ½ lb) --- Food:rôti de

## Analyse des résultats

Sur le dataset de test, nous obtenons un résultat de 118/126 pour les quantitées et de 96/126 pour les ingrédients.

### Quantités

Pour les quantités, certaines erreurs semblent être des erreurs du jeu de test. Par exemple, nous obtenons 1 cuillère à soupe (15 ml) alors que la réponse selon le jeu de test est cuillère à soupe (15 ml). Le 1 est manquant, ce qui est surprenant puisque le 1 devrait normalement se retrouver dans la réponse. En ce qui concerne les autres erreurs, nous n'avons pas capturé au goût ou facultatif, ce qui serait possible en ajustant la capture du terme. Cependant, plus la regex contient de terme et plus elle devient complexe et difficile à débogger. Certaines quantités sont plus complexes et demanderait un regex beaucoup plus complexe pour le récupérer tel que "3 d’environ 340 g (¾ lb) chacun" et "1 rôti de 675 g (1 ½ lb)". Ces cas pourraient être ajoutés au regex, cependant, seulement les cas des tests seraient capturés alors que plusieurs autres cas qui ne sont pas dans le jeu de données ne le seraient pas.

### Ingrédients

Contrairement aux quantités, les ingrédients n'ont pas de règles spécifiques, ce qui rend le développement d'une regex difficile. Nous avons choisi de capturer la première séquence de un à trois termes avec le premier terme ayant au moins 3 lettres suivant les quantités. Selon nos tests, c'est le nombre de mots qui donnait le meilleur résultat. Capturer seulement les séquences débutant avec un mot d'au moins 3 lettres sert à éviter de capturer les termes tel que "d'" et "de" se situant devant les ingrédients, mais conserver ces mêmes mots lorsqu'ils sont dans la séquence, par exemple, pour la séquence d'huile d'olive, le premier "d'" est retiré, mais le deuxième est conservé, ce qui nous donne "huile d'olive".

## Section 5 - Section réservée pour notre correction

Ne pas retirer les cellules de cette section. 